# ONNX Runtime 库介绍

ONNX Runtime 和 ONNX 是两个相关但功能不同的库：

- ONNX 库的功能是定义神经网络模型的标准格式，支持跨框架模型导出和转换，专注于模型的设计、定义和优化。
- ONNX Runtime 库功能是高性能推理引擎，用于执行和优化 ONNX 格式的模型，专注于在多种硬件上高效运行 ONNX 模型。  
    
如此设计拆分的原因有：

- 职责分离: ONNX 负责模型定义，ONNX Runtime 负责模型执行，各自专注提高性能。
- 灵活性: 拆分后，开发者可以灵活选择工具，适应不同需求。
- 独立发展: 允许两个库独立更新和优化，满足各自领域的技术发展。

# 加载 ONNX 模型

In [1]:
! wget https://labfile.oss.aliyuncs.com/courses/40981/multi_io_model.onnx

--2025-03-22 14:20:21--  https://labfile.oss.aliyuncs.com/courses/40981/multi_io_model.onnx
Resolving labfile.oss.aliyuncs.com (labfile.oss.aliyuncs.com)... 47.110.177.159
Connecting to labfile.oss.aliyuncs.com (labfile.oss.aliyuncs.com)|47.110.177.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71256 (70K) [application/octet-stream]
Saving to: 'multi_io_model.onnx'

     0K .......... .......... .......... .......... .......... 71% 9.01M 0s
    50K .......... .........                                  100% 1.08M=0.02s

2025-03-22 14:20:21 (2.94 MB/s) - 'multi_io_model.onnx' saved [71256/71256]



In [1]:
import onnxruntime as ort 

model_path = 'multi_io_model.onnx'
session  = ort.InferenceSession(model_path)
# ort.InferenceSession 用于创建一个推理会话，该会话将载入指定路径下的 ONNX 模型。


# 查看模型输入输出格式


In [ ]:
input_details = [input_ for input_ in session.get_inputs()]
for idx,input_detail in enumerate(input_details):
     print(f"第{idx+1}个输入形状：{input_detail.shape}, 数据类型：{input_detail.type}, 输入名称：{input_detail.name}")


第1个输入形状：['batch_size', 10], 数据类型：tensor(float), 输入名称：input1
第2个输入形状：['batch_size', 20], 数据类型：tensor(float), 输入名称：input2
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x000001BA0E4E0DF0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x000001BA0E4E07B0>]


因为我们事先并不知道一个模型是单输入还是多输入，所以需要遍历所有输入，依次打印每个输入的形状和数据类型。

- input_details.shape 返回输入的形状，它是一个列表或元组，表示每个维度的大小。
- input_details.type 返回输入的数据类型。

通过打印结果，我们可以看到这个模型需要输入两个 float 张量，节点 input1 尺寸为 ['batch_size', 10]，另一个 input2 为 ['batch_size', 20]

同样的，获取输出格式，只需要把 .get_inputs 换成 .get_outputs 即可

In [4]:
output_details = [input_ for input_ in session.get_outputs()]
for idx, output_detail in enumerate(output_details):
    print(f"第{idx+1}个输出形状：{output_detail.shape}, 数据类型：{output_detail.type}, 输出名称：{output_detail.name}")

第1个输出形状：['batch_size', 1], 数据类型：tensor(float), 输出名称：output1
第2个输出形状：['batch_size', 2], 数据类型：tensor(float), 输出名称：output2


# 模型推理

In [7]:
import numpy as np
input_data1 = np.random.rand(3,10).astype(np.float32)
input_data2 = np.random.rand(3,20).astype(np.float32)

outputs = session.run(None,{"input1":input_data1,"input2":input_data2})
print(outputs)

[array([[ 0.02000945],
       [-0.03383649],
       [-0.02177634]], dtype=float32), array([[-0.00301798, -0.20539439],
       [ 0.03358935, -0.16653925],
       [ 0.12497212, -0.15285939]], dtype=float32)]


input_data1 和 input_data2 是为模型提供的随机输入数据，形状分别为 (3, 10) 和 (3, 20)。
session.run 函数执行模型推理，None 表示获取所有输出。输入名称 "input1"，"input2" 需要与模型的输入名称匹配。

In [8]:
output1 = session.run(['output1'], {"input1": input_data1, "input2": input_data2})
output2 = session.run(['output2'], {"input1": input_data1, "input2": input_data2})
print(f'{output1=}\n{output2=}')

output1=[array([[ 0.02000945],
       [-0.03383649],
       [-0.02177634]], dtype=float32)]
output2=[array([[-0.00301798, -0.20539439],
       [ 0.03358935, -0.16653925],
       [ 0.12497212, -0.15285939]], dtype=float32)]


# 获取中间结果

In [9]:
import copy
import onnx
onnx_model = onnx.load('multi_io_model.onnx')
# 深拷贝模型的输出信息，ori_output 保存原始模型输出的结构，防止后续修改影响原有模型
ori_output = copy.deepcopy(onnx_model.graph.output)

for node in onnx_model.graph.node:
    for output in node.output:
         onnx_model.graph.output.extend([onnx.ValueInfoProto(name=output)])

通过循环遍历模型的每个节点，提取每个节点的输出，并将其添加到模型的 output 列表中，这样模型在推理时不仅会输出最终的结果，还会输出每一层的中间结果。

然后我们为修改后的 ONNX 模型创建推理会话，并进行推理，使用有序字典顺序存储输出结果

In [10]:
from collections import OrderedDict

# 使用 ONNX Runtime 创建推理会话，SerializeToString 将模型序列化为字节流
ort_session = ort.InferenceSession(onnx_model.SerializeToString())
# 运行推理，使用 input_data1 和 input_data2 作为模型的输入
ort_outs = ort_session.run(None, {"input1": input_data1, "input2": input_data2})

# 获取模型中所有输出节点的名称
outputs = [x.name for x in ort_session.get_outputs()]

# 将输出的名称和对应的输出值以有序字典的形式组合，便于查找
ort_outs = OrderedDict(zip(outputs, ort_outs))
print(ort_outs)

OrderedDict([('output1', array([[ 0.02000945],
       [-0.03383649],
       [-0.02177634]], dtype=float32)), ('output2', array([[-0.00301798, -0.20539439],
       [ 0.03358935, -0.16653925],
       [ 0.12497212, -0.15285939]], dtype=float32)), ('/Constant_output_0', array(0.5, dtype=float32)), ('/Mul_output_0', array([[0.42464387, 0.47286236, 0.22205007, 0.02009887, 0.40676284,
        0.22793245, 0.36089095, 0.46112078, 0.4685927 , 0.45767406],
       [0.41848215, 0.02656515, 0.41577333, 0.43224296, 0.10359064,
        0.34651738, 0.17580359, 0.32082206, 0.40584224, 0.00789164],
       [0.20272185, 0.44108552, 0.495985  , 0.44066486, 0.48080394,
        0.22549027, 0.41936097, 0.3724126 , 0.36917216, 0.11440618]],
      dtype=float32)), ('/fc1/Gemm_output_0', array([[-0.05027089, -0.28285018, -0.10937285, ..., -0.4459226 ,
         0.2738302 ,  0.55757916],
       [-0.15196466, -0.15361848, -0.04035488, ..., -0.43479687,
         0.40952694,  0.36501145],
       [-0.16973707, -0.20146